#Common PATHWAYS obtained with the different methods of functional enrichment

In [0]:
#Load libraries
## Append the library folder
.libPaths(c("/dbfs/home/boriol@almirall.com/my_r_packages/bulkRNASeq_PBMCs_R4.3", .libPaths()))

library(DESeq2)
library(tidyverse)
library(VennDiagram)
library(RColorBrewer)

.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat", .libPaths()))
library(openxlsx)
library(enrichplot)
library(ggplot2)

#KC

In [0]:
Dif_KC_GSVA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSVA/Differentiated_KC_GSVA_res.sig.xlsx")
Dif_KC_GSVA$pathway_name <- gsub("_", " ", Dif_KC_GSVA$pathway_name)
Dif_KC_GSVA$gs_description <- Dif_KC_GSVA$pathway_name

Undif_KC_GSVA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSVA/Undifferentiated_KC_GSVA_res.sig.xlsx")
Undif_KC_GSVA$pathway_name <- gsub("_", " ", Undif_KC_GSVA$pathway_name)
Undif_KC_GSVA$gs_description <- Undif_KC_GSVA$pathway_name

Dif_KC_GSEA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/res_dif_kc_0.05_AR.xlsx" , sheet="Reactome")
Undif_KC_GSEA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/res_undif_kc_0.05_AR.xlsx" , sheet="Reactome")

Dif_KC_GSEA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/dif_kc_reactome_genes.xlsx")
Undif_KC_GSEA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/undif_kc_reactome_genes.xlsx")

In [0]:
max(Dif_KC_GSEA$p.adjust)

In [0]:
# Function to check if at least 80% of the words are equal and don't count the first word in GSVA
have_common_words <- function(str1, str2, min_common_percentage = 0.8) {
  words1 <- unlist(strsplit(tolower(str1), "\\s+|\\-"))[-1]  # Remove the first word and split by space or hyphen
  words2 <- unlist(strsplit(tolower(str2), "\\s+|\\-"))  # Split by space or hyphen
  common_words <- intersect(words1, words2)
  return(length(common_words) / length(words1) >= min_common_percentage)
}

# Create a list of your gene sets
gene_sets <- list(
  "GSVA" = Dif_KC_GSVA,
  "GSEA" = Dif_KC_GSEA
)

# Find common pathways based on the custom criteria
common_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), core_degs = character(), stringsAsFactors = FALSE)
contrary_sign_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), core_degs = character(), stringsAsFactors = FALSE)
only_gsva_pathways <- data.frame(Pathway = character(), logFC = numeric(), stringsAsFactors = FALSE)

for (i in 1:nrow(gene_sets$GSVA)) {
  found_common <- FALSE
  for (j in 1:nrow(gene_sets$GSEA)) {
    pathway1 <- gene_sets$GSVA$pathway_name[i]
    pathway2 <- gene_sets$GSEA$Description[j]
    if (have_common_words(pathway1, pathway2)) {
      found_common <- TRUE
      if (sign(gene_sets$GSVA$logFC[i]) == sign(gene_sets$GSEA$NES[j])) {
        common_pathways <- rbind(common_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j], core_degs = gene_sets$GSEA$core_degs[j]))
      } else {
        contrary_sign_pathways <- rbind(contrary_sign_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j], core_degs = gene_sets$GSEA$core_degs[j]))
      }
      break
    }
  }
  if (!found_common) {
    only_gsva_pathways <- rbind(only_gsva_pathways, data.frame(Pathway = pathway1, logFC = gene_sets$GSVA$logFC[i]))
  }
}

# Print common pathways and their logFC, NES, and core_degs
display(common_pathways)


In [0]:
display(only_gsva_pathways)

In [0]:
# Function to check if at least 80% of the words are equal and don't count the first word in GSVA
have_common_words <- function(str1, str2, min_common_percentage = 0.8) {
  words1 <- unlist(strsplit(tolower(str1), "\\s+"))[-1]  # Remove the first word
  words2 <- unlist(strsplit(tolower(str2), "\\s+"))
  common_words <- intersect(words1, words2)
  return(length(common_words) / length(words1) >= min_common_percentage)
}

# Create a list of your gene sets
gene_sets <- list(
  "GSVA" = Undif_KC_GSVA,
  "GSEA" = Undif_KC_GSEA
)

# Find common pathways based on the custom criteria
common_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), core_degs = character(), stringsAsFactors = FALSE)
contrary_sign_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), core_degs = character(), stringsAsFactors = FALSE)

for (i in 1:nrow(gene_sets$GSVA)) {
  for (j in 1:nrow(gene_sets$GSEA)) {
    pathway1 <- gene_sets$GSVA$pathway_name[i]
    pathway2 <- gene_sets$GSEA$Description[j]
    if (have_common_words(pathway1, pathway2)) {
      if (sign(gene_sets$GSVA$logFC[i]) == sign(gene_sets$GSEA$NES[j])) {
        common_pathways <- rbind(common_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j], core_degs = gene_sets$GSEA$core_degs[j]))
      } else {
        contrary_sign_pathways <- rbind(contrary_sign_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j], core_degs = gene_sets$GSEA$core_degs[j]))
      }
      break
    }
  }
}

# Print common pathways and their logFC, NES, and core_degs
display(common_pathways)


In [0]:
display(contrary_sign_pathways)


#KC

In [0]:
Dif_KC_GSVA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSVA/Differentiated_KC_GSVA_res.sig.xlsx")
Dif_KC_GSVA$pathway_name <- gsub("_", " ", Dif_KC_GSVA$pathway_name)
Dif_KC_GSVA$gs_description <- Dif_KC_GSVA$pathway_name

Undif_KC_GSVA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSVA/Undifferentiated_KC_GSVA_res.sig.xlsx")
Undif_KC_GSVA$pathway_name <- gsub("_", " ", Undif_KC_GSVA$pathway_name)
Undif_KC_GSVA$gs_description <- Undif_KC_GSVA$pathway_name

Dif_KC_GSEA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/Limma/res_kc_0.05_AR.xlsx" , sheet="Reactome")
Undif_KC_GSEA <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/Limma/res_undifkc_0.05_AR.xlsx" , sheet="Reactome")

In [0]:
max(Dif_KC_GSEA$p.adjust)

In [0]:
# Function to check if at least 80% of the words are equal and don't count the first word in GSVA
have_common_words <- function(str1, str2, min_common_percentage = 0.8) {
  words1 <- unlist(strsplit(tolower(str1), "\\s+"))[-1]  # Remove the first word
  words2 <- unlist(strsplit(tolower(str2), "\\s+"))
  common_words <- intersect(words1, words2)
  return(length(common_words) / length(words1) >= min_common_percentage)
}

# Create a list of your gene sets
gene_sets <- list(
  "GSVA" = Dif_KC_GSVA,
  "GSEA" = Dif_KC_GSEA
)

# Find common pathways based on the custom criteria
common_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), stringsAsFactors = FALSE)
contrary_sign_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), stringsAsFactors = FALSE)

for (i in 1:nrow(gene_sets$GSVA)) {
  for (j in 1:nrow(gene_sets$GSEA)) {
    pathway1 <- gene_sets$GSVA$pathway_name[i]
    pathway2 <- gene_sets$GSEA$Description[j]
    if (have_common_words(pathway1, pathway2)) {
      if (sign(gene_sets$GSVA$logFC[i]) == sign(gene_sets$GSEA$NES[j])) {
        common_pathways <- rbind(common_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j]))
      } else {
        contrary_sign_pathways <- rbind(contrary_sign_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j]))
      }
      break
    }
  }
}

# Print common pathways and their logFC and NES
display(common_pathways)

In [0]:
display(contrary_sign_pathways)

In [0]:
# Function to check if at least 80% of the words are equal and don't count the first word in GSVA
have_common_words <- function(str1, str2, min_common_percentage = 0.8) {
  words1 <- unlist(strsplit(tolower(str1), "\\s+"))[-1]  # Remove the first word
  words2 <- unlist(strsplit(tolower(str2), "\\s+"))
  common_words <- intersect(words1, words2)
  return(length(common_words) / length(words1) >= min_common_percentage)
}

# Create a list of your gene sets
gene_sets <- list(
  "GSVA" = Undif_KC_GSVA,
  "GSEA" = Undif_KC_GSEA
)

# Find common pathways based on the custom criteria
common_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), stringsAsFactors = FALSE)
contrary_sign_pathways <- data.frame(Pathway = character(), logFC = numeric(), NES = numeric(), stringsAsFactors = FALSE)

for (i in 1:nrow(gene_sets$GSVA)) {
  for (j in 1:nrow(gene_sets$GSEA)) {
    pathway1 <- gene_sets$GSVA$pathway_name[i]
    pathway2 <- gene_sets$GSEA$Description[j]
    if (have_common_words(pathway1, pathway2)) {
      if (sign(gene_sets$GSVA$logFC[i]) == sign(gene_sets$GSEA$NES[j])) {
        common_pathways <- rbind(common_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j]))
      } else {
        contrary_sign_pathways <- rbind(contrary_sign_pathways, data.frame(Pathway = pathway1, Pathway2 = pathway2, logFC = gene_sets$GSVA$logFC[i], NES = gene_sets$GSEA$NES[j]))
      }
      break
    }
  }
}

# Print common pathways and their logFC, NES, and core_degs
display(common_pathways)


In [0]:
display(contrary_sign_pathways)
